## LOAD SUBREDDITS

In [1]:
import glob

json_files = glob.glob('./data/*.json')
print(json_files)

import json

def getRedditSubreddits(json_files):
  subreddits = {}
  for json_file in json_files:
    with open(json_file) as f:
      data = json.load(f)
      firstKey = next(iter(data))
      subredditName = data[firstKey]['subreddit']
      subreddits[subredditName] = data
  return subreddits

def getTopComments(subreddit_name, post_id):
  # ./data/subreddit_name/post_id_top_comments.json
  json_file = './data/' + subreddit_name + '/' + post_id + '_top_comments.json'
  with open(json_file) as f:
    data = json.load(f)
    return data

def saveredditSubreddits(subreddits):
  for subreddit in subreddits:
    with open('./data/' + subreddit + '_top_posts.json', 'w') as outfile:
      json.dump(subreddits[subreddit], outfile, indent=4)

subreddits = getRedditSubreddits(json_files)

print("subreddits", subreddits.keys())


['./data/facepalm_top_posts.json', './data/announcements_top_posts.json', './data/DunderMifflin_top_posts.json', './data/wholesomememes_top_posts.json', './data/science_top_posts.json', './data/BikiniBottomTwitter_top_posts.json', './data/freefolk_top_posts.json', './data/MadeMeSmile_top_posts.json', './data/StarWarsBattlefront_top_posts.json', './data/books_top_posts.json', './data/tifu_top_posts.json', './data/nextfuckinglevel_top_posts.json', './data/EarthPorn_top_posts.json', './data/gifs_top_posts.json', './data/AmItheAsshole_top_posts.json', './data/todayilearned_top_posts.json', './data/me_irl_top_posts.json', './data/WatchPeopleDieInside_top_posts.json', './data/thanosdidnothingwrong_top_posts.json', './data/Showerthoughts_top_posts.json', './data/Wellthatsucks_top_posts.json', './data/wallstreetbets_top_posts.json', './data/nottheonion_top_posts.json', './data/comics_top_posts.json', './data/PewdiepieSubmissions_top_posts.json', './data/awfuleverything_top_posts.json', './data

## Cull Posts without Comments

In [2]:
total_deleted = 0
for subreddit in subreddits:
  deleted = 0
  for post_id in reversed(list(subreddits[subreddit])):
    #print("post_id", post_id)
    try:
      top_comments = getTopComments(subreddit, post_id)
    except:
      # remove post_id from subreddits[subreddit]
      del subreddits[subreddit][post_id]
      print("post_id", post_id, "removed")
      deleted += 1
      continue
  if deleted > 0:
    print("subreddit", subreddit, "deleted", deleted, "posts")
    total_deleted += deleted
if total_deleted > 0:
  saveredditSubreddits(subreddits)

In [15]:
def diveInPost(subreddit_name, post_id):
  global commentPathsTemp
  topComments = getTopComments(subreddit_name, post_id)
  #print(subreddit_name, post_id, len(topComments))
  commentPathsTemp = []
  commentTree = []
  for comment_id in topComments:
    comment = topComments[comment_id]
    #print(comment['body'])
    diveRecursion(topComments, comment_id, [])
    commentPaths = commentPathsTemp.copy()
    #print(commentPaths)
    commentPathsTemp = []
    commentTree.append(commentPaths)
  #displayCommentTree(commentTree, topComments)
  return topComments, commentTree
def is_nested(array):
  return any(isinstance(i, list) for i in array)

def normalizeNestedArray(array):
  if not is_nested(array):
    return array
  else:
    return array[0]

commentPathsTemp = []
# Base case: comment has no other comments listing it as a parent_id
#   - add comment to commentArray then return commentArray
# Recursive Case: add comment to commentArray, and call diveRecursion on all comments that list it as a parent 
def diveRecursion(topComments, comment_id, commentArray):
  global commentPathsTemp
  comment = topComments[comment_id]
  commentArray.append(comment_id)
  # loop through every comment checking if it has a parent_id of comment_id
  count = 0
  for other_comment_id in topComments:
    if topComments[other_comment_id]['parent_id'] == f"t1_{comment_id}":
      diveRecursion(topComments, other_comment_id, commentArray.copy())
      count += 1
  if count == 0 and len(commentArray) > 0: # base case, no other comments list this comment as a parent. We're at a leaf.
    commentPathsTemp.append(commentArray.copy())

def displayCommentTree(commentTree, topComments, indent=0):
  for commentPath in commentTree:
    for comment_id, i in zip(commentPath, range(len(commentPath))):
      comment = topComments[comment_id]
      indentStr = ' ' * i
      print(indentStr + comment['id'])
      
    print('\n')


In [17]:
def normalizeUpvotes(max_upvotes, upvotes):
  # normalize upvotes to be between 0 and 1 with 2 decimal places
  return round(upvotes / max_upvotes, 2)
def removeDeletedComments(commentPath, topComments):
  # remove deleted comments from commentPath
  #print("commentPath", commentPath)
  for comment_id in reversed(list(commentPath)):
    comment = topComments[comment_id]
    #print(comment['body'])
    is_suspended = comment['author']['is_suspended']
      
    if is_suspended or comment['author']['author_name'].strip() == '[deleted]' or comment['body'].strip() == '[deleted]' or comment['body'].strip() == '[removed]':
      #print('deleted')
      commentPath.remove(comment_id)
  return commentPath

In [5]:
for subreddit_name in subreddits:
  #print(subreddit_name)
  for post_id in subreddits[subreddit_name]:
    #print(post_id ,subreddits[subreddit_name][post_id]['title'])
    topComments, commentTree = diveInPost(subreddit_name, post_id)

# commentTree is an array of commentPaths. Each commentPath is an array of comment_ids.


facepalm
gupyuy Cops pepper sprayed their own Senator without realizing he's an authority figure
10cfs82 german riot police defeated and humiliated by some kind of mud wizard
13org0v The healthcare system in America is awful.
ubwjys Amber Heard's lawyer objecting to his own question
13n65xa She knew exactly what was Happening 
sh6ppv Math is a Myth.
m4xual The state of the world.
giaf19 Scientific name = poison
i91c5f the USPS is not a business
13fh2y6 YouTuber is facing 20 years in prison after deliberately crashing a plane for views.
announcements
7jsyqt The FCC’s vote was predictably frustrating, but we’re not done fighting for net neutrality.
8jzyb6 Update: We won the Net Neutrality vote in the Senate!
80jsi9 Upvote the Downvote: Tell Congress to use the CRA to save net neutrality!
3cucye An old team at reddit
8i3382 (Orange)Red Alert: The Senate is about to vote on whether to restore Net Neutrality
mcisdf An update on the recent issues surrounding a Reddit employee
hrrh23 Now you 

In [18]:
# remove deleted comments from commentPath
for subreddit_name in subreddits:
  for post_id in subreddits[subreddit_name]:
    topComments, commentTree = diveInPost(subreddit_name, post_id)
    for commentPaths in commentTree:
      
      #topComments = getTopComments(subreddit_name, post_id)

      for commentPath in commentPaths:
        for comment_id in commentPath:        
          removeDeletedComments(commentPath, topComments)

In [19]:
# create jsonl file just for subreddit aww
# https://huggingface.co/blog/llama2#how-to-prompt-llama-2
jsonl = []

for post_id in subreddits[subreddit_name]:
  topComments, commentTree = diveInPost(subreddit_name, post_id)
  for commentPaths in commentTree:
    
    #topComments = getTopComments(subreddit_name, post_id)
    # get max upvotes for this post
    #first key in topComments 
    firstKey = next(iter(topComments))
    max_upvotes = topComments[firstKey]['score']
    
    for commentPath in commentPaths:

      commentPath = removeDeletedComments(commentPath, topComments)
      if len(commentPath) == 0:
        continue
      #commentPath = commentPath[:3] # only use the first comment in the path TEST

      _instruction = f"""<s>[INST] <<SYS>>
You are a Reddit user comment generator.
In the conversation you change your Reddit username often to simulate different users.
<</SYS>>

You are on the subreddit /r/{subreddit_name}.
You want to post a comment in the post titled: {subreddits[subreddit_name][post_id]['title']}
The post media type is: {subreddits[subreddit_name][post_id]['classification']}
The post submission is about: {subreddits[subreddit_name][post_id]['text']}
The Original Poster(OP) username is: {subreddits[subreddit_name][post_id]['author_name']}
Your username is made up by you.
Each reply is in the format USERNAME(UPVOTES_RATIO): REPLY [/INST]"""
      _input = "Respond to the post. </s><s>"
      # Loop through commentPath and add every other comment to instruction
      # Every other comment in the path needs to be wrapped in [INST] [/INST] to simulate conversation for llama
      if len(commentPath) > 1:
        for comment_id, i in zip(commentPath[:-1], range(len(commentPath[:-1]))):
          comment = topComments[comment_id]
          upvote_ratio = normalizeUpvotes(max_upvotes, comment['score'])
          reply = f"{comment['author']['author_name']}({upvote_ratio}): {comment['body'].strip()}"
          _input += f"""{reply} </s><s>"""
        _input += "</s>"
        # if instruction ends with <s></s> remove it
        if _input.endswith("<s></s>"): # happens when commentPath is odd
          _input = _input[:-7]
      else:
        # remove <s> from end of _input
        _input = _input[:-3]

      # output is the last comment in the path
      # This simulates the expected output of the model
      lastComment = topComments[commentPath[-1]]
      upvote_ratio = normalizeUpvotes(max_upvotes, lastComment['score'])
      _output = f"<s> {lastComment['author']['author_name']}({upvote_ratio}): {lastComment['body'].strip()} </s>"
      
      jsonl_template = {
        "instruction": _instruction,
        "input": _input,
        "output": _output,
        "text": f"{_instruction} {_input}{_output} "
      }
      
      jsonl.append(jsonl_template)
      #print(jsonl_template['text'])
      
    


In [20]:
with open('reddit_comments.jsonl', 'w') as outfile:
  for entry in jsonl:
    outfile.write(json.dumps(entry) + '\n')